Imports

In [1]:
import os
import re
import json
import fitz  
from PIL import Image
from io import BytesIO
from urllib.parse import urlparse
import spacy
import google.generativeai as genai
from dotenv import load_dotenv
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
nlp = spacy.load("en_core_web_sm")

/opt/anaconda3/envs/langchain_setup_llama/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Configure Gemini

In [2]:
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)
text_model = genai.GenerativeModel("gemini-2.0-flash")
vision_model = genai.GenerativeModel("gemini-2.0-flash")

Rubric

In [3]:
rubric = {
    "architect_chosen": 5,
    "doc_and_slides": 5,
    "bio_750_words": 5,
    "bio_structure": 5,
    "bio_references": 5,
    "10_buildings_with_images": 5,
    "image_quality": 5,
    "image_citations": 5,
    "image_relevance": 5,
    "personal_bio_photo": 5,
    "presentation_polish": 5
}

rubric_descriptions = {
    "architect_chosen": "Is the architect selected from Book Two and clearly identified?",
    "doc_and_slides": "Is the document structured well with table of contents and all required sections?",
    "bio_750_words": "Does the biography meet the 750-word requirement?",
    "bio_structure": "Does the biography cover who they are, where they studied, etc.?",
    "bio_references": "Are there 5–10 APA references with DOIs and citation counts?",
    "10_buildings_with_images": "Are 10 buildings covered with names, locations, significance, and image suggestions?",
    "image_quality": "Are the images high-resolution and well-composed?",
    "image_citations": "Do all images have proper attribution (photographer/source)?",
    "image_relevance": "Do images clearly relate to the architect’s work?",
    "personal_bio_photo": "Is a professional student photo and 1–2 sentence bio included?",
    "presentation_polish": "Is the document polished, well-formatted, and web-publishable?"
}

 Extract text from PDF

In [4]:
pdf_path = "/Users/tanishqsingh/Desktop/XR_Lab/cogs160submisson1.pdf"

In [5]:
def extract_text_from_pdf(pdf_path):
    print(f" Extracting text from: {pdf_path}")
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text()
    print(" Extracted text from PDF")
    return text

 Extract images from PDF

In [6]:
def extract_images_from_pdf(pdf_path, min_width=1200, save_folder="/Users/tanishqsingh/Desktop/XR_Lab/Extracted_images"):
    print(f" Extracting images from: {pdf_path}")
    doc = fitz.open(pdf_path)
    os.makedirs(save_folder, exist_ok=True)
    image_data = []
    for page_index in range(len(doc)):
        page = doc[page_index]
        images = page.get_images(full=True)
        for img_index, img in enumerate(images):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            img_pil = Image.open(BytesIO(image_bytes))
            width, height = img_pil.size
            img_pil.save(os.path.join(save_folder, f"page{page_index+1}_img{img_index+1}.png"))
            image_data.append({
                "page": page_index + 1,
                "width": width,
                "height": height,
                "coordinates": img[1:5],
                "image": img_pil,
                "filename": f"page{page_index+1}_img{img_index+1}.png",
                "is_high_res": width >= min_width
            })
    print(f" Extracted {len(image_data)} images")
    return image_data

Match Image with Nearby Text

In [7]:
def get_caption_candidates(text, image_data):
    print("Scanning for image captions...")
    lines = text.split("\n")
    results = []

    for img in image_data:
        context = {
            "page": img["page"],
            "image": img["filename"],
            "matched_caption": "",
            "has_citation": False,
            "has_building_name": False,
            "has_interior_note": False
        }
        for i, line in enumerate(lines):
            if f"{img['filename'].split('.')[0]}" in line:
                nearby_lines = lines[max(i-2, 0): i+3]
                caption_text = " ".join(nearby_lines)
                context["matched_caption"] = caption_text
                context["has_citation"] = any(x in caption_text.lower() for x in ["source", "http", "photographer"])
                context["has_building_name"] = bool(re.search(r"(building|tower|museum|villa|house|center)", caption_text, re.IGNORECASE))
                context["has_interior_note"] = bool(re.search(r"(interior|lobby|hall|inside)", caption_text, re.IGNORECASE))
                break
        results.append(context)
    return results

Evaluate Image Relevance, Building Match, and Attribution via Gemini

In [8]:
def evaluate_images_with_gemini(image_data, architect_name, debug=False):
    print(" Evaluating image content and relevance using Gemini...")
    enriched_image_feedback = []

    for img in tqdm(image_data, desc="Evaluating images"):
        prompt = f"""
You are reviewing an image submitted for a university architecture project about {architect_name}.
Please analyze this image and answer:

1. Does this image show a building designed by {architect_name}? If yes, specify the building.
2. Is this an interior or exterior shot?
3. Does this image clearly show architectural features (e.g., lighting, geometry, layout)?
4. How relevant is this image for an academic project about {architect_name}?

Give your feedback in the following JSON format:
{{
  "building_detected": "...",
  "interior_or_exterior": "...",
  "relevance_score": "x/10",
  "justification": "...",
  "architectural_features_visible": true/false
}}
"""
        try:
            response = vision_model.generate_content([img["image"], prompt])
            if debug:
                print(f"Image {img['filename']} feedback:\n", response.text)
            cleaned_text = response.text.strip()
            if cleaned_text.startswith("```"):
                cleaned_text = re.sub(r"```(?:json)?", "", cleaned_text)
                cleaned_text = cleaned_text.replace("```", "").strip()

            try:
                data = json.loads(cleaned_text)
            except Exception as e:
                print(f" Still failed to parse JSON from {img['filename']}: {e}")
                data = {
                    "building_detected": "Unknown",
                    "interior_or_exterior": "Unknown",
                    "relevance_score": "5/10",
                    "justification": "Could not parse feedback from Gemini.",
                    "architectural_features_visible": False
                }
        except Exception as e:
            print(f"⚠️ Error processing {img['filename']}: {e}")
            data = {
                "building_detected": "Unknown",
                "interior_or_exterior": "Unknown",
                "relevance_score": "5/10",
                "justification": "Could not extract structured feedback.",
                "architectural_features_visible": False
            }

        data.update({
            "filename": img["filename"],
            "page": img["page"],
            "width": img["width"],
            "height": img["height"],
            "is_high_res": img["is_high_res"]
        })
        enriched_image_feedback.append(data)
    return enriched_image_feedback

Score Image Citations and Structure Compliance

In [9]:
def evaluate_image_structure_and_captions(image_feedback, caption_context):
    print("Evaluating caption presence and structure")

    scores = []
    per_image_feedback = []

    for img in image_feedback:
        caption = next((c for c in caption_context if c["image"] == img["filename"]), {})
        has_citation = caption.get("has_citation", False)
        has_building_name = caption.get("has_building_name", False)
        has_interior_note = caption.get("has_interior_note", False)

        score = 0
        if has_citation: score += 3
        if has_building_name: score += 3
        if has_interior_note: score += 2
        if img["is_high_res"]: score += 2

        scores.append(score)
        per_image_feedback.append({
            "image": img["filename"],
            "page": img["page"],
            "relevance_score": img.get("relevance_score", "5/10"),
            "justification": img.get("justification", ""),
            "caption_found": caption.get("matched_caption", ""),
            "has_proper_caption": score >= 7,
            "score": score
        })

    avg_score = sum(scores) / len(scores) if scores else 0
    return {
        "score": int((avg_score / 10) * rubric["image_citations"]),
        "details": per_image_feedback
    }

Gemini-Based Rubric Evaluation with Chain-of-Thought

In [10]:
def gemini_detailed_rubric_eval(text, architect_name):
    print(" Gemini evaluating full rubric with explanations")

    prompt = f"""
You are evaluating a student's architecture assignment on the architect {architect_name}.

This is a formal submission for university credit. You are receiving the full document as **images**, so you can directly observe the formatting, embedded images, captions, structure, and layout.

---

###  How to Grade:

- Be **fair and constructive**. If formatting is inconsistent, information is missing, or citations are weak, **please call it out clearly**.
- Do not sugarcoat — students are expected to revise based on your feedback.
- If something is strong, note it. If it's flawed, critique it.
- When scoring, **prioritize**:
  - Accuracy of academic citations
  - Caption and image attribution clarity
  - Clear distinction between interior vs exterior images
  - Overall layout and visual professionalism

---

###  Additional Clarifications:

-  Images are embedded (not just links)
-  Captions below images include attribution (URLs or photographer names)
-  A student photo and bio appear on Page 2
-  Table of Contents is present
-  10 buildings are described
-  Redundant links are likely citations, not missing content

---

###  RUBRIC CRITERIA

Please assess each of the following categories. For every criterion:

1. Give a **detailed justification** (1–2 paragraphs)
2. Assign a score **out of 5** based on the detailed rubric below

Format:
**[Category Name]**
Justification: ...
Score: x/5

---

###  Categories and Rubric Anchors

**1. Architect Selection & Scope**
- 5 = Clearly identifies one architect from Book Two, explicitly stated, on-topic
- 3–4 = Identifies Book Two architect, but clarity or justification could improve
- 1–2 = Architect unclear, off-topic, or not from Book Two

**2. Organization & Document Setup**
- 5 = Clear Table of Contents + labeled sections for bio, buildings, refs, student bio
- 3–4 = Minor issues with layout or missing headers
- 1–2 = Poor organization, missing sections, or hard to follow

**3. Biographical Content (750 words)**
- 5 = Covers who they are, achievements, education, significance, 1st building, typologies
- 3–4 = Mostly complete, with slight omissions or light detail
- 1–2 = Underdeveloped or below word count, missing major points

**4. Citation of Architect Biography**
- 5 = 5–10 academic references, correct APA formatting, includes DOIs and citation counts
- 3–4 = APA errors or missing DOIs, but still academically relevant
- 1–2 = Few or no academic references, poor or irrelevant sources

**5. Selection & Quality of Images**
- 5 = 10 buildings, 3+ exterior + 5+ interior per building, high-res
- 3–4 = Most buildings meet criteria; a few lack resolution or quantity
- 1–2 = Many buildings missing images or poor quality

**6. Image Citation & Attribution**
- 5 = Every image has clear, consistent source or photographer citation
- 3–4 = Most are cited but with some inconsistencies
- 1–2 = Citations mostly missing, inconsistent, or improperly formatted

**7. Coverage of 10 Famous Buildings**
- 5 = All 10 named + location + significance statement (1–2 sentences)
- 3–4 = Buildings listed but some lack significance or location
- 1–2 = Several missing or incomplete building descriptions

**8. Image Relevance**
- 5 = All images relate directly to described buildings, match descriptions, show architectural value
- 3–4 = Most images relevant, some generic or misaligned
- 1–2 = Several images are off-topic or not associated with described buildings

**9. Personal Bio & Photo**
- 5 = Professional photo and bio (1–2 sentences), correctly placed after TOC
- 3–4 = Present but minor formatting/image issues
- 1–2 = Photo or bio is low quality, misplaced, or absent

**10. Overall Completeness & Presentation**
- 5 = Fully polished, clean layout, minimal repetition, suitable for web/publication
- 3–4 = Clear submission, but lacks design polish or has formatting repetition
- 1–2 = Sloppy or rushed presentation; visual issues hurt readability

---

 Please start your rubric-based analysis below:
"""

    doc = fitz.open(pdf_path)
    all_pages_as_images = [page.get_pixmap(dpi=300).pil_tobytes("png") for page in doc]
    response = vision_model.generate_content(
         [prompt] + [Image.open(BytesIO(img_bytes)) for img_bytes in all_pages_as_images]
      )

    print(response.text)

    def extract_score(label, out_of):
        match = re.search(label + r".*?Score:\s*(\d+)/" + str(out_of), response.text, re.IGNORECASE | re.DOTALL)
        return int(match.group(1)) if match else 0

    return {
        "architect_chosen": {"score": extract_score("Architect Selection", 5)},
        "doc_and_slides": {"score": extract_score("Organization", 5)},
        "bio_750_words": {"score": extract_score("Biographical Content", 5)},
        "bio_structure": {"score": extract_score("Biographical Structure", 5)},
        "bio_references": {"score": extract_score("Citation of Architect Biography", 5)},
        "10_buildings_with_images": {"score": extract_score("Coverage of 10 Famous Buildings", 5)},
        "image_quality": {"score": extract_score("Selection & Quality of Images", 5)},
        "image_citations": {"score": extract_score("Image Citation & Attribution", 5)},
        "image_relevance": {"score": extract_score("Image Relevance", 5)},
        "personal_bio_photo": {"score": extract_score("Personal Bio", 5)},
        "presentation_polish": {"score": extract_score("Presentation Polish", 5)},
    }

Final Aggregator + Full Diagnostic Report

In [11]:
def generate_detailed_scorecard(scores, image_caption_details=None):
    print(" Compiling final scorecard")

    # Total and max only for defined rubric keys
    total = sum([scores[k]["score"] for k in scores if k in rubric])
    max_total = sum([rubric[k] for k in scores if k in rubric])
    final_percentage = (total / max_total) * 100 if max_total else 0

    grade = "A" if final_percentage >= 50 else "B" if final_percentage >= 46 else "C" if final_percentage >= 42 else "D"

    # print(f"Final Grade: {grade} ({round(final_percentage, 2)}%)")
    rubric_table = pd.DataFrame([
        {
            "Criterion": k.replace("_", " ").title(),
            "Score": scores[k]["score"],
            "Max": rubric[k],
            "Description": rubric_descriptions.get(k, "")
        }
        for k in rubric if k in scores
    ])
    display(rubric_table)
    if image_caption_details:
        print("\n Image Caption & Relevance Feedback:")
        df = pd.DataFrame(image_caption_details["details"])
        display(df)

    return {
        "rubric_scores": {k: scores[k]["score"] for k in rubric if k in scores},
        "final_percent": round(final_percentage, 2),
        "grade": grade,
        "image_feedback_table": image_caption_details
    }

In [12]:
def extract_references_from_text(text):
    print(" Extracting references from text")
    lines = text.split("\n")
    references = []
    for line in lines:
        if re.search(r"\(\d{4}\)", line) and any(x in line.lower() for x in ["doi", "archdaily", "e-architect", "https://", "http://"]):
            references.append(line.strip())
    return references


In [13]:
def evaluate_biography(text):
    print(" Evaluating biography: checking word count and required sections")
    result = {}
    doc = nlp(text)
    result["word_count"] = len([token.text for token in doc if token.is_alpha])

    required_sections = [
        "who they are",
        "famous for",
        "studied",
        "significance",
        "influence",
        "types of buildings",
        "first building"
    ]

    section_hits = sum([1 for section in required_sections if section.lower() in text.lower()])
    result["structure_score"] = int((section_hits / len(required_sections)) * rubric["bio_structure"])
    result["score"] = rubric["bio_750_words"] if result["word_count"] >= 700 else int((result["word_count"] / 750) * rubric["bio_750_words"])

    return result


In [14]:
def evaluate_image_quality(image_data):
    print(" Evaluating image resolution")
    high_res_count = sum(1 for img in image_data if img["is_high_res"])
    total_images = len(image_data)
    
    quality_score = int((high_res_count / max(1, total_images)) * rubric["image_quality"])
    
    print(f" {high_res_count}/{total_images} images are high resolution")
    return {"high_res_count": high_res_count, "score": quality_score}

Main Runner Pipeline

In [15]:
def run_autograder_full(pdf_path, architect_name="Bjarke Ingels", debug=False):
    print("Starting full grading pipeline")
    text = extract_text_from_pdf(pdf_path)
    images = extract_images_from_pdf(pdf_path)
    references = extract_references_from_text(text)
    image_caption_context = get_caption_candidates(text, images)
    enriched_feedback = evaluate_images_with_gemini(images, architect_name, debug)
    image_caption_score = evaluate_image_structure_and_captions(enriched_feedback, image_caption_context)
    gemini_scores = gemini_detailed_rubric_eval(text, architect_name)
    manual_scores = {}
    renamed_gemini = {
        "architect_chosen": gemini_scores["architect_chosen"],
        "doc_and_slides": gemini_scores["doc_and_slides"],
        "personal_bio_photo": gemini_scores["personal_bio_photo"],
        "presentation_polish": gemini_scores["presentation_polish"]
    }
    all_scores = {**manual_scores, **renamed_gemini}
    result = generate_detailed_scorecard(all_scores, image_caption_score)
    return result

Run 

In [16]:
result = run_autograder_full("/Users/tanishqsingh/Desktop/XR_Lab/cogs160submisson1.pdf", "Bjarke Ingels", debug=True)

Starting full grading pipeline
 Extracting text from: /Users/tanishqsingh/Desktop/XR_Lab/cogs160submisson1.pdf
 Extracted text from PDF
 Extracting images from: /Users/tanishqsingh/Desktop/XR_Lab/cogs160submisson1.pdf
 Extracted 79 images
 Extracting references from text
Scanning for image captions...
 Evaluating image content and relevance using Gemini...


Evaluating images:   1%|▏         | 1/79 [00:02<02:38,  2.03s/it]

Image page2_img1.png feedback:
 ```json
{
  "building_detected": "No",
  "interior_or_exterior": "N/A",
  "relevance_score": "1/10",
  "justification": "The image shows a person wearing a blazer outdoors, with a background of trees and a clear sky. It does not depict any architecture, let alone a building designed by Bjarke Ingels. Therefore, it is not relevant to an academic project on him.",
  "architectural_features_visible": false
}
```


Evaluating images:   3%|▎         | 2/79 [00:04<02:38,  2.05s/it]

Image page3_img1.png feedback:
 ```json
{
  "building_detected": "Unclear, but potentially relates to a building by Bjarke Ingels. The design of the structure behind the man appears to be influenced by LEGO bricks and could be a feature of LEGO House in Billund, Denmark, designed by BIG (Bjarke Ingels Group).",
  "interior_or_exterior": "Exterior",
  "relevance_score": "7/10",
  "justification": "The image depicts Bjarke Ingels himself, which directly relates to the project. The background possibly showcasing features from one of his designs adds further relevance. However, the image primarily features Bjarke Ingels; the architectural design is secondary, impacting the relevance score. Without explicit confirmation of the building, it loses relevance.",
  "architectural_features_visible": true
}
```


...

Image page51_img2.png feedback:
 {"building_detected": "Uncertain",
  "interior_or_exterior": "Interior",
  "relevance_score": "6/10",
  "justification": "The image showcases a modern interior with exposed ductwork, geometric wall patterns, and integrated lighting which align with Bjarke Ingels Group's (BIG) style. However, without knowing the building, I can't confirm it is one of his works. The space appears to be a gym, so, if confirmed to be one of his buildings, it shows an element of his design philosophy. The clarity is excellent, allowing for detailed observation of the interior elements.",
  "architectural_features_visible": true
}
Evaluating caption presence and structure
 Gemini evaluating full rubric with explanations


Okay, I will provide a detailed rubric-based analysis of the provided document.

**1. Architect Selection & Scope**
Justification: The student has clearly identified Bjarke Ingels, who is definitely an architect from Book Two. The assignment is on-topic and focused on his work.
Score: 5/5

**2. Organization & Document Setup**
Justification: The document has a clear Table of Contents. It includes sections for the student's bio, personal information about Ingels, 10 famous buildings, a biography, and academic references. The order of sections is slightly off. The "Biography" should come before the "10 Famous Buildings".
Score: 4/5

**3. Biographical Content (750 words)**
Justification: The biography appears to cover the necessary points: who Bjarke Ingels is, his achievements, education, and significance. It also briefly touches on his first major credited building and typologies. While it's hard to confirm the exact word count from images, it seems reasonably comprehensive, though sligh

,Criterion,Score,Max,Description
0,Architect Chosen,5,5,Is the architect selected from Book Two and cl...
1,Doc And Slides,4,5,Is the document structured well with table of ...
2,Personal Bio Photo,5,5,Is a professional student photo and 1–2 senten...
3,Presentation Polish,0,5,"Is the document polished, well-formatted, and ..."



 Image Caption & Relevance Feedback:


,image,page,relevance_score,justification,caption_found,has_proper_caption,score
0,page2_img1.png,2,1/10,The image shows a person wearing a blazer outd...,,False,0
1,page3_img1.png,3,7/10,"The image depicts Bjarke Ingels himself, which...",,False,0
2,page4_img1.png,4,9/10,The image clearly showcases the architectural ...,,False,2
3,page5_img1.png,5,9/10,The image shows the distinctive architectural ...,,False,2
4,page5_img2.png,5,10/10,The image clearly depicts the exterior of the ...,,False,2
...,...,...,...,...,...,...,...
74,page49_img1.png,49,5/10,Could not parse feedback from Gemini.,,False,2
75,page50_img1.png,50,3/10,The image shows a modern office space with int...,,False,2
76,page50_img2.png,50,8/10,The image showcases the biophilic design princ...,,False,2
77,page51_img1.png,51,9/10,The image showcases the interior of the Google...,,False,2
